# TF-IDF(Term Frequency-Inverse Document Frequency)

## 출처: [4) TF-IDF(Term Frequency-Inverse Document Frequency)](https://wikidocs.net/31698)

## 1. 단어 빈도-역 문서 빈도

- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- DTM 생성 후 TF-IDF 가중치 부여
- 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 사용
- TF와 IDF를 곱한 값을 의미
- 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때,   
1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수,   
2) df(t) : 특정 단어 t가 등장한 문서의 수,   
3) idf(d, t) : df(t)에 반비례하는 수로 정의 가능

## 2. TF-IDF 직접 구현하기

### 01. 라이브러리 추가

In [1]:
import pandas as pd # 데이터프레임 사용
from math import log # IDF 계산

In [2]:
# 문서 정의
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 

In [3]:
vocab = list(set(w for doc in docs for w in doc.split()))

In [4]:
vocab.sort()

### 02. TF, IDF, TF-IDF 값 구하는 함수 구현

In [5]:
# 총 문서의 수
N = len(docs)

In [6]:
# TF 값 정의
def tf(t, d):
    return d.count(t)

In [7]:
# IDF 값 정의
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

In [8]:
# TF-IDF 값 구하는 함수 정의
def tfidf(t, d):
    return tf(t,d)* idf(t)

### 03. DTM을 데이터프레임에 저장하여 출력

In [9]:
result = []

In [10]:
# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

In [11]:
tf_ = pd.DataFrame(result, columns = vocab)

In [12]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### 04. 각 단어에 대한 IDF 값 구하기

In [13]:
result2 = []

In [14]:
for j in range(len(vocab)):
    t = vocab[j]
    result2.append(idf(t))

In [16]:
idf_ = pd.DataFrame(result2, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


### 05. TF-IDF 행렬 출력

In [17]:
result3 = []

In [18]:
for i in range(N):
    result3.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result3[-1].append(tfidf(t,d))

In [19]:
tfidf_ = pd.DataFrame(result3, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## 3. 사이킷런을 이용한 DTM과 TF-IDF 실습

### 01. DTM 생성

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# Corpus 생성
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

In [22]:
# CountVectorizer 정의
vector = CountVectorizer()

In [23]:
# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]


In [24]:
# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### 02. 사이킷런에서 제공하는 TF-IDF를 자동 계산해주는 TfidfVectorizer 사용

In [25]:
# 위의 보편적인 TF-IDF 기본 식에서 조정된 식을 사용
# IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정
# TF-IDF가 가진 의도는 여전히 그대로 갖고 있음

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Corpus 생성
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

In [28]:
# TfidfVectorizer 정의 
tfidfv = TfidfVectorizer().fit(corpus)

In [29]:
# 코퍼스로부터 각 단어의 빈도수를 기록
print(tfidfv.transform(corpus).toarray())

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]


In [30]:
# 각 단어와 맵핑된 인덱스 출력
print(tfidfv.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
